# Aplicação do Dropout na rede neural para verificar viabilidade

In [1]:
import pandas as pd
import tensorflow as tf
import sklearn

In [2]:
from tensorflow.keras.layers import Dense, Dropout, Activation, Input
from tensorflow.keras.models import Model # Utilizaremos a Model ao invés de Sequential
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [3]:
# https://www.kaggle.com/datasets/gregorut/videogamesales
base = pd.read_csv('games.csv')
base

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# Removendo colunas que não serão utilizadas
base = base.drop('Other_Sales', axis = 1)
base = base.drop('Global_Sales', axis = 1)
base = base.drop('Developer', axis = 1)

In [6]:
base.shape

(16719, 13)

# Processamento da base de dados

In [5]:
base.isnull().sum()

Name                  2
Platform              0
Year_of_Release     269
Genre                 2
Publisher            54
NA_Sales              0
EU_Sales              0
JP_Sales              0
Critic_Score       8582
Critic_Count       8582
User_Score         6704
User_Count         9129
Rating             6769
dtype: int64

In [7]:
base = base.dropna(axis = 0) # Removendo linhas com valores nulos

In [8]:
base.shape # Reduziu de 16.719 para 6825 linhas (não é muito recomendado remover linhas, ainda mais que representam 59% dos dados)

(6825, 13)

In [9]:
base.isnull().sum()

Name               0
Platform           0
Year_of_Release    0
Genre              0
Publisher          0
NA_Sales           0
EU_Sales           0
JP_Sales           0
Critic_Score       0
Critic_Count       0
User_Score         0
User_Count         0
Rating             0
dtype: int64

In [10]:
base['Name'].value_counts()

Name
Need for Speed: Most Wanted                  8
Madden NFL 07                                8
LEGO Star Wars II: The Original Trilogy      8
The Sims 2                                   7
Terraria                                     7
                                            ..
Castlevania: Portrait of Ruin                1
Suzuki TT Superbikes                         1
Rumble Roses                                 1
Sherlock Holmes: The Mystery of the Mummy    1
STORM: Frontline Nation                      1
Name: count, Length: 4377, dtype: int64

In [11]:
base = base.drop('Name', axis = 1) # Iremos remover a coluna Name, pois ela não é útil para o modelo

In [12]:
X = base.iloc[:, [0, 1, 2, 3, 7, 8, 9, 10, 11]].values # Features ou atributos previsores
X

array([['Wii', 2006.0, 'Sports', ..., '8', 322.0, 'E'],
       ['Wii', 2008.0, 'Racing', ..., '8.3', 709.0, 'E'],
       ['Wii', 2009.0, 'Sports', ..., '8', 192.0, 'E'],
       ...,
       ['PC', 2014.0, 'Action', ..., '7.6', 412.0, 'M'],
       ['PC', 2011.0, 'Shooter', ..., '5.8', 43.0, 'T'],
       ['PC', 2011.0, 'Strategy', ..., '7.2', 13.0, 'E10+']], dtype=object)

In [13]:
y_na = base.iloc[:, 4].values # Atributo a serem previstos
y_eu = base.iloc[:, 5].values
y_jp = base.iloc[:, 6].values

In [14]:
# PS2  1 0 0 0 0 ...
# X360 0 1 0 0 0 ...
base['Platform'].value_counts()

Platform
PS2     1140
X360     858
PS3      769
PC       651
XB       565
Wii      479
DS       464
PSP      390
GC       348
PS4      239
GBA      237
XOne     159
3DS      155
PS       150
PSV      118
WiiU      89
DC        14
Name: count, dtype: int64

In [15]:
base.columns

Index(['Platform', 'Year_of_Release', 'Genre', 'Publisher', 'NA_Sales',
       'EU_Sales', 'JP_Sales', 'Critic_Score', 'Critic_Count', 'User_Score',
       'User_Count', 'Rating'],
      dtype='object')

In [16]:
onehotencoder = ColumnTransformer(transformers=[("OneHot", OneHotEncoder(), [0, 2, 3, 8])], remainder='passthrough') # Passamos as colunas CATEGÓRICAS para o OneHotEncoder
X = onehotencoder.fit_transform(X).toarray()

In [17]:
X.shape # 6825 linhas e 303 colunas

(6825, 303)

In [18]:
X[0]

array([0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       1.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 1.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
       0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 

In [19]:
# Criação da rede neural a partir da biblioteca Model
camada_entrada = Input(shape = (303,)) # Quantidade de neurônios é sempre igual a quantidade de colunas (303)
camada_oculta1 = Dense(units = 153, activation='relu')(camada_entrada) # Adicionamos uma camada oculta com 153 neurônios conectada à camada de entrada
camada_dropout1 = Dropout(0.2)(camada_oculta1) # Adicionamos uma camada de Dropout para evitar overfitting
camada_oculta2 = Dense(units = 153, activation='relu')(camada_oculta1)
camada_dropout2 = Dropout(0.2)(camada_oculta2)
camada_saida1 = Dense(units = 1, activation='linear')(camada_oculta2) # As camadas de saída estarão conectadas à última camada oculta (camada_oculta2)
camada_saida2 = Dense(units = 1, activation='linear')(camada_oculta2)
camada_saida3 = Dense(units = 1, activation='linear')(camada_oculta2)

In [20]:
regressor = Model(inputs = camada_entrada, outputs = [camada_saida1, camada_saida2, camada_saida3])

In [21]:
regressor.compile(optimizer='adam', loss = 'mse') # A função de loss mse (mean squared error)

In [25]:
regressor.fit(X, [y_na, y_eu, y_jp], epochs=500, batch_size=100)

Epoch 1/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 1.0688 - dense_3_loss: 0.8748 - dense_4_loss: 0.1219 - loss: 2.0641
Epoch 2/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 0.6499 - dense_3_loss: 0.6273 - dense_4_loss: 0.1287 - loss: 1.4057
Epoch 3/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 1.1657 - dense_3_loss: 0.7175 - dense_4_loss: 0.3003 - loss: 2.1832
Epoch 4/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 1.0467 - dense_3_loss: 0.9300 - dense_4_loss: 0.1850 - loss: 2.1622
Epoch 5/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 1.4998 - dense_3_loss: 1.2516 - dense_4_loss: 0.1176 - loss: 2.8693
Epoch 6/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 0.8107 - dense_3_loss: 0.5425 - dense_4_loss: 0.1539 - loss: 1.5074
Epoch 7/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_2_loss: 1.1429 - dense_3_loss: 0.9379 - dense_4_loss: 0.1644 - loss: 2.2456
Epoch 8/500
69/69 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - de

In [26]:
previsao_na, previsao_eu, previsao_jp = regressor.predict(X) # Testando o modelo com os próprios dados de treinamento

214/214 ━━━━━━━━━━━━━━━━━━━━ 0s 977us/step


# Previsão com o Dropout de 20%

In [38]:
previsao_na, previsao_na.mean(), y_na, y_na.mean()

(array([[ 8.381581  ],
        [ 9.650656  ],
        [ 6.082191  ],
        ...,
        [-0.012909  ],
        [-0.1028443 ],
        [-0.29132205]], dtype=float32),
 0.3234921,
 array([4.136e+01, 1.568e+01, 1.561e+01, ..., 0.000e+00, 1.000e-02,
        0.000e+00]),
 0.3944835164835165)

In [39]:
previsao_eu, previsao_eu.mean(), y_eu, y_eu.mean()

(array([[ 5.9325094e+00],
        [ 6.6482086e+00],
        [ 4.2632928e+00],
        ...,
        [ 3.7254691e-03],
        [-4.3580413e-02],
        [-1.4271954e-01]], dtype=float32),
 0.19636914,
 array([2.896e+01, 1.276e+01, 1.093e+01, ..., 1.000e-02, 0.000e+00,
        1.000e-02]),
 0.23608937728937732)

In [40]:
previsao_jp, previsao_jp.mean(), y_jp, y_jp.mean()

(array([[ 2.6732123e+00],
        [ 2.7076676e+00],
        [ 1.9526049e+00],
        ...,
        [-1.6678572e-03],
        [-1.7802827e-02],
        [-5.1616915e-02]], dtype=float32),
 0.05936809,
 array([3.77, 3.79, 3.28, ..., 0.  , 0.  , 0.  ]),
 0.06415824175824175)

In [29]:
from sklearn.metrics import mean_absolute_error

# Erro absoluto com o Dropout

In [30]:
mean_absolute_error(y_na, previsao_na)

0.260387293560601

In [41]:
mean_absolute_error(y_eu, previsao_eu)

0.18426966578209353

In [42]:
mean_absolute_error(y_jp, previsao_jp)

0.08722818011471203

# Erro absoluto sem o Dropout

In [35]:
mean_absolute_error(y_na, previsao_na)

0.3225100472481696

In [38]:
mean_absolute_error(y_eu, previsao_eu)

0.22454100766815108

In [42]:
mean_absolute_error(y_jp, previsao_jp)

0.1189478561075179